In [6]:
import tensorflow as tf
print('piyush Kumar')
print('test')

tf.__version__

piyush Kumar
test


'2.13.0'

In [2]:
import os
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [38]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
   
    def save_model(self,path: Path, model: tf.keras.Model, epoch_path: Path):
        model.save(path)
        last_epoch =  str(self.config.params_epochs)
        f =open(epoch_path, "w+")
        f.write(last_epoch)
        f.close()



    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model = tf.keras.models.load_model("model/model.h5")
        f = open("epochs.txt", "r")
        initial_epoch = int(f.read())
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            initial_epoch = initial_epoch
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model,
            epoch_path= Path("epochs.txt")

        ) 

In [39]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-02-23 18:54:11,160: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-23 18:54:11,176: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-23 18:54:11,176: INFO: common: created directory at: artifacts]
[2024-02-23 18:54:11,184: INFO: common: created directory at: artifacts\training]
Found 131 images belonging to 2 classes.
Found 529 images belonging to 2 classes.
Epoch 42/42
33/33 [==============================] - 63s 2s/step - loss: 0.0236 - accuracy: 0.9903 - val_loss: 0.3142 - val_accuracy: 0.9141


d:\projects\Kidney-Disease-Classification-Deep-Learning-Project\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [40]:
f = open("epochs.txt", "r")
int(f.read())

42

In [46]:
if os.path.getsize("epochs.txt") != 0:
    f = open("epochs.txt", "r")
    initial_epoch = int(f.read())
else:
    initial_epoch = 0

In [50]:
os.path.getsize("artifacts/training/model.h5") != 0

True

In [1]:
from cnnClassifier import logger

In [2]:
initial_epoch = 10

In [5]:
logger.info(f"***** training started from epoch   {initial_epoch+1}+1 ******")


[2024-02-25 12:33:41,453: INFO: 3557137719: ***** training started from epoch   11+1 ******]


In [7]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/piyushsharma161/Kidney-Disease-Classification-Deep-Learning-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="piyushsharma161"
os.environ["MLFLOW_TRACKING_PASSWORD"]="7ca0fdeb8edc864f6a98ffc6112a41e276d078fe"